In [ ]:
# @title 1. Setup & Installation

print(" Installing optimized stack \n")
# We use sdpa (built-in), so no need for flash-attn pip install
!pip install -q -U torch transformers==4.37.0 bitsandbytes accelerate  datasets huggingface_hub tqdm scikit-learn matplotlib seaborn pandas safetensors

print(" Installation complete!\n")
print("✅ Installation complete!\n")

# Verify installation
print("📦 Verifying package versions:")
!pip show torch transformers bitsandbytes accelerate | grep "Name:\|Version:"

In [ ]:
# @title Load Environment Variables
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()
login(token=os.getenv("HUGGING_FACE_HUB_TOKEN"))

In [ ]:
# @title Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import json
import random
from tqdm.auto import tqdm
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import os
import gc
import platform

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
    # BitsAndBytesConfig removed: Not compatible with MPS
)
from datasets import load_dataset
from huggingface_hub import HfApi, create_repo, login
from safetensors.torch import save_file

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# 1. Global Determinism Anchor (MPS Compatible)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

if torch.backends.mps.is_available():
    # MPS does not currently support a global manual_seed_all like CUDA
    # but setting the manual_seed covers the generator.
    torch.mps.manual_seed(SEED)

# 2. Device Detection Logic
if torch.backends.mps.is_available():
    device = torch.device("mps")
    precision_mode = "Float16 (MPS Optimized)"
    compute_dtype = torch.float16
elif torch.cuda.is_available():
    device = torch.device("cuda")
    precision_mode = "Float16 (CUDA)"
    compute_dtype = torch.float16
else:
    device = torch.device("cpu")
    precision_mode = "Float32 (CPU Fallback)"
    compute_dtype = torch.float32

# 3. Environment Forensic Dashboard
print("="*80)
print(" FAIRSTEER RESEARCH SUITE: APPLE SILICON EDITION")
print("="*80)
print(f" System OS:       {platform.system()} {platform.release()}")
print(f" Processor:       {platform.processor()}")
print(f" Active Device:   {device.type.upper()}")
print(f" Logic Precision: {precision_mode}")

if device.type == "mps":
    # Note: MPS does not provide direct VRAM 'total' queries through torch yet.
    # It shares the system's Unified Memory.
    print(" Architecture:    Unified Memory Architecture (UMA)")
    print(" Performance:     Metal Performance Shaders (MPS) Active")

print("="*80 + "\n")

In [ ]:
# @title 3. Configuration (Dream-7B Diffusion Edition)

import torch
from typing import List, Optional

class TrainingConfig:
    # --- MODEL & ARCHITECTURE (Dream-7B/LLaDA Specific) ---
    # Dream-v0 is a bidirectional Transformer diffusion model
    base_model_name = "Dream-org/Dream-v0-Instruct-7B"
    model_hidden_dim = 4096
    max_seq_len = 1024 # Standard context window for extraction
    num_layers = 32
    
    # --- DIFFUSION PARAMETERS ---
    # Extraction at t=0.5 (Maximum semantic ambiguity, ideal for BAD probe training)
    extraction_t = 0.5 
    # Dream-7B Tokenizer Specifics
    # We will fetch this dynamically in the loader, but standard is 126348 or similar
    mask_token_id = None 
    
    # --- DATASET CONFIGURATION (FairSteer Standard) ---
    bbq_dataset_name = "bitlabsdb/BBQ_dataset"
    bbq_target_loc_dataset = "bitlabsdb/bbq_target_loc_dedup"
    MMLU_DATASET = "bitlabsdb/MMLU"
    BBQA_DATASET = "bitlabsdb/BBQA"
    
    # Manifold Sizes (Scaled for 7B VRAM limits)
    num_bbq_samples = 20 
    mmlu_data_size = 4   
    DSV_TARGET = 110       
    
    # --- HARDWARE & PIPELINE OPTIMIZATION ---
    train_val_split = 0.8
    batch_size = 32
    # LLaDA/Dream forward passes are memory-heavy due to bidirectional attention
    # We reduce batch size to ensure FlashAttention/SDPA stability
    extraction_batch_size = 4 
    SEED = 42
    
    # Full Layer Scanning for the "Diffusion Causal Bottleneck"
    candidate_layers_range = list(range(0, 32))
    
    # --- LABELS (FairSteer Aligned) ---
    LABEL_BIASED = 0
    LABEL_UNBIASED = 1

    # --- SAVE & DEPLOYMENT ---
    hf_repo_name = "bitlabsdb/Dream-7B-FairSteer-BAD"
    local_save_dir = "./artifacts_dream_7b"
    # Dream-v0 Instruct uses specific prompt templates (Algorithm 2)
    with_header = True 
    
    IS_DEBUG = False

    @property
    def model_id_short(self) -> str:
        return self.base_model_name.split("/")[-1]

config = TrainingConfig()

# Global Environment Forensic Dashboard
print("="*80)
print(" 🛡️ CONFIGURATION - DREAM-7B DIFFUSION RESEARCH SUITE")
print("="*80)
print(f"   • Model Path:         {config.base_model_name}")
print(f"   • Architecture:       Masked Diffusion (Bidirectional)")
print(f"   • Logic Precision:    {precision_mode}")
print(f"   • Extraction Strategy: Token-wise Masked Pooling at t={config.extraction_t}")
print(f"   • GPU Optimization:   SDPA / FlashAttention Active")
print("="*80 + "\n")

In [ ]:
# @title 5. BBQ Dataset Integrated Merging (Composite Key Integrity) - Merged by example_id & category
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset
import warnings

def load_and_merge_bbq(config) -> pd.DataFrame:
    """
    Loads BBQ and Targets, merges via Composite Key, and performs config-driven
    sub-sampling to output the final training manifold.
    """

    print("="*80)
    print(" 🚀 RESEARCH-GRADE BBQ LOADER & SUB-SAMPLER (COMPOSITE KEY VERSION)")
    print("="*80 + "\n")

    # ═══════════════════════════════════════════════════════════════════════════
    # 1. DATA ACQUISITION
    # ═══════════════════════════════════════════════════════════════════════════
    print("1. Loading Primary BBQ Dataset...")
    try:
        # Use config-defined path if available
        ds_name = getattr(config, 'bbq_dataset_name', "bitlabsdb/BBQ_dataset")
        bbq_ds = load_dataset(ds_name, split="train")
    except Exception as e:
        print(f"   ⚠️ Primary loading failed: {e}. Attempting fallback...")
        bbq_ds = load_dataset("bitlabsdb/BBQ_dataset", split="train")

    df_bbq = pd.DataFrame(bbq_ds)
    df_bbq['example_id'] = pd.to_numeric(df_bbq['example_id'], errors='coerce').fillna(-1).astype(int)
    print(f"   ✅ Primary BBQ Loaded: {len(df_bbq):,} rows.")

    # ═══════════════════════════════════════════════════════════════════════════
    # 2. METADATA PREPARATION (Stereotype Targets)
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n2. Loading Target Locations (Stereotype Metadata)...")
    try:
        loc_ds = load_dataset("bitlabsdb/bbq_target_loc_dedup", split="train")
    except:
        # Fallback to local script if needed
        loc_ds = load_dataset("bitlabsdb/bbq_target_loc_dedup", split="train")

    df_loc = pd.DataFrame(loc_ds)
    df_loc['example_id'] = pd.to_numeric(df_loc['example_id'], errors='coerce').dropna().astype(int)
    df_loc['target_loc'] = pd.to_numeric(df_loc['target_loc'], errors='coerce')
    df_loc = df_loc[df_loc['target_loc'].isin([0, 1, 2])]
    df_loc['target_loc'] = df_loc['target_loc'].astype(int)

    # Deduplicate on Composite Key (ID + Category) to ensure 1:1 mapping
    df_loc = df_loc.drop_duplicates(subset=['example_id', 'category'], keep='first')
    print(f"   ✅ Target Metadata Prepared: {len(df_loc):,} unique causal pairs.")

    # ═══════════════════════════════════════════════════════════════════════════
    # 3. COMPOSITE MERGE & INTEGRITY AUDIT
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n3. Executing Composite Merge & Integrity Audit...")
    integrity_check = pd.merge(
        df_bbq,
        df_loc[['example_id', 'category', 'target_loc']],
        on=['example_id', 'category'],
        how='left',
        indicator=True
    )

    df_merged = integrity_check[integrity_check['_merge'] == 'both'].drop(columns=['_merge']).copy()
    df_missing = integrity_check[integrity_check['_merge'] == 'left_only'].copy()

    count_total = len(df_bbq)
    count_merged = len(df_merged)

    # ═══════════════════════════════════════════════════════════════════════════
    # 4. CONFIG-DRIVEN SUB-SAMPLING (THE "FAIRSTEER" FILTER)
    # ═══════════════════════════════════════════════════════════════════════════
    if hasattr(config, 'num_bbq_samples') and config.num_bbq_samples is not None:
        if config.num_bbq_samples < count_merged:
            print(f"\n✂️  APPLYING SUB-SAMPLING: Filtering to {config.num_bbq_samples:,} samples...")
            # We sample deterministically using config.SEED to maintain research reproducibility
            df_final = df_merged.sample(n=config.num_bbq_samples, random_state=config.SEED).copy()
        else:
            print(f"\nℹ️  Config limit ({config.num_bbq_samples:,}) exceeds available merged data. Using all merged records.")
            df_final = df_merged.copy()
    else:
        print("\nℹ️  No sub-sampling limit found in config. Using full merged dataset.")
        df_final = df_merged.copy()

    count_final = len(df_final)

    # ═══════════════════════════════════════════════════════════════════════════
    # 5. RESEARCH DASHBOARD
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n📊 Generating Research Dashboard...")
    sns.set_theme(style="whitegrid")
    fig, axes = plt.subplots(1, 3, figsize=(22, 7))
    plt.suptitle(f"BBQ Data Curation Pipeline (Final N={count_final:,})", fontsize=18, weight='bold', y=1.02)

    # Plot 1: Attrition Flow
    retention_labels = ['Total Input', 'Valid Merged', 'Config Final']
    retention_values = [count_total, count_merged, count_final]
    sns.barplot(x=retention_labels, y=retention_values, palette='Blues_r', ax=axes[0], edgecolor='black')
    axes[0].set_title("Data Retention Flow", fontsize=14, weight='bold')
    for i, v in enumerate(retention_values):
        axes[0].text(i, v + (count_total * 0.02), f"{v:,}", ha='center', weight='bold')

    # Plot 2: Final Categorical Distribution
    sns.countplot(
        y='category',
        data=df_final,
        order=df_final['category'].value_counts().index,
        palette="viridis",
        ax=axes[1],
        edgecolor='black'
    )
    axes[1].set_title(f"Final Category Mix (N={count_final:,})", fontsize=14, weight='bold')

    # Plot 3: Context Distribution (Ambig vs Disambig)
    df_final['context_condition'].value_counts().plot.pie(
        autopct='%1.1f%%', colors=['#74b9ff', '#fab1a0'], ax=axes[2],
        startangle=140, wedgeprops={'edgecolor':'black'}
    )
    axes[2].set_ylabel('')
    axes[2].set_title("Context Composition", fontsize=14, weight='bold')

    plt.tight_layout()
    plt.show()

    print("\n" + "="*80)
    print(f"💎 FINAL MANIFOLD READY: {count_final:,} records assigned to 'bbq_merged_df'")
    print("="*80 + "\n")

    return df_final

# Execute the pipeline and populate the global variable
bbq_merged_df = load_and_merge_bbq(config)

In [ ]:
# @title 5.2 BBQA Dataset Integrated Merging (Composite Key Integrity)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset
import warnings

def load_and_merge_bbqa(config) -> pd.DataFrame:
    """
    OpenAI/MIT Standard: Manifold Curation Engine.

    Loads the refined BBQA dataset and merges with stereotype target metadata
    using composite key integrity (example_id + category).
    """

    print("="*80)
    print(" 🚀 RESEARCH-GRADE BBQ LOADER: REFINED MANIFOLD EDITION")
    print("="*80 + "\n")

    # ═══════════════════════════════════════════════════════════════════════════
    # 1. DATA ACQUISITION: Refined BBQA
    # ═══════════════════════════════════════════════════════════════════════════
    print("1. Loading Primary BBQA Dataset [bitlabsdb/BBQA]...")
    try:
        # Targeting the specific refined repo as requested
        bbq_ds = load_dataset("bitlabsdb/BBQA", split="train")
    except Exception as e:
        print(f"   ❌ CRITICAL: Primary loading failed: {e}")
        return None

    df_bbq = pd.DataFrame(bbq_ds)

    # Forensic Type Casting: Ensure example_id is a strictly typed integer
    df_bbq['example_id'] = pd.to_numeric(df_bbq['example_id'], errors='coerce').fillna(-1).astype(int)
    print(f"   ✅ Primary BBQA Loaded: {len(df_bbq):,} rows.")

    # ═══════════════════════════════════════════════════════════════════════════
    # 2. METADATA PREPARATION: Stereotype Targets
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n2. Loading Target Locations [bitlabsdb/bbq_target_loc_dedup]...")
    try:
        loc_ds = load_dataset("bitlabsdb/bbq_target_loc_dedup", split="train")
    except Exception as e:
        print(f"   ❌ CRITICAL: Target metadata loading failed: {e}")
        return None

    df_loc = pd.DataFrame(loc_ds)

    # Normalizing metadata keys
    df_loc['example_id'] = pd.to_numeric(df_loc['example_id'], errors='coerce').dropna().astype(int)
    df_loc['target_loc'] = pd.to_numeric(df_loc['target_loc'], errors='coerce')

    # Causal Guard: Filter for valid choice indices (A=0, B=1, C=2)
    df_loc = df_loc[df_loc['target_loc'].isin([0, 1, 2])]
    df_loc['target_loc'] = df_loc['target_loc'].astype(int)

    # Deduplicate on Composite Key to ensure 1:1 causal mapping
    df_loc = df_loc.drop_duplicates(subset=['example_id', 'category'], keep='first')
    print(f"   ✅ Target Metadata Prepared: {len(df_loc):,} unique causal pairs.")

    # ═══════════════════════════════════════════════════════════════════════════
    # 3. COMPOSITE MERGE: Semantic Alignment
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n3. Executing Composite Merge & Integrity Audit...")

    # We use a LEFT join on both ID and Category to preserve semantic context
    integrity_check = pd.merge(
        df_bbq,
        df_loc[['example_id', 'category', 'target_loc']],
        on=['example_id', 'category'],
        how='left',
        indicator=True
    )

    # Identify successful manifold alignments
    df_merged = integrity_check[integrity_check['_merge'] == 'both'].drop(columns=['_merge']).copy()

    count_total = len(df_bbq)
    count_merged = len(df_merged)

    # ═══════════════════════════════════════════════════════════════════════════
    # 4. CONFIG-DRIVEN SUB-SAMPLING
    # ═══════════════════════════════════════════════════════════════════════════
    if hasattr(config, 'num_bbq_samples') and config.num_bbq_samples is not None:
        if config.num_bbq_samples < count_merged:
            print(f"\n✂️  APPLYING SUB-SAMPLING: Filtering to {config.num_bbq_samples:,} samples...")
            df_final = df_merged.sample(n=config.num_bbq_samples, random_state=config.SEED).copy()
        else:
            print(f"\nℹ️  Config limit exceeds available data. Using all {count_merged:,} records.")
            df_final = df_merged.copy()
    else:
        df_final = df_merged.copy()

    count_final = len(df_final)

    # ═══════════════════════════════════════════════════════════════════════════
    # 5. RESEARCH DASHBOARD: Manifold Distribution
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n📊 Visualizing Final Training Manifold...")
    sns.set_theme(style="whitegrid")
    fig, axes = plt.subplots(1, 3, figsize=(22, 7))
    plt.suptitle(f"BBQA Data Curation Pipeline (N={count_final:,})", fontsize=18, weight='bold', y=1.02)

    # Attrition Flow
    retention_labels = ['Input (BBQA)', 'Valid Merged', 'Sub-sampled']
    retention_values = [count_total, count_merged, count_final]
    sns.barplot(x=retention_labels, y=retention_values, palette='Blues_r', ax=axes[0], edgecolor='black')
    axes[0].set_title("Data Retention Flow", fontsize=14, weight='bold')

    # Category Mix
    sns.countplot(
        y='category',
        data=df_final,
        order=df_final['category'].value_counts().index,
        palette="viridis",
        ax=axes[1],
        edgecolor='black'
    )
    axes[1].set_title("Categorical Saturation", fontsize=14, weight='bold')

    # Ambig vs Disambig
    df_final['context_condition'].value_counts().plot.pie(
        autopct='%1.1f%%', colors=['#74b9ff', '#fab1a0'], ax=axes[2],
        startangle=140, wedgeprops={'edgecolor':'black'}
    )
    axes[2].set_title("Context Composition", fontsize=14, weight='bold')

    plt.tight_layout()
    plt.show()

    print("\n" + "="*80)
    print(f"💎 FINAL MANIFOLD READY: {count_final:,} records assigned to 'bbq_merged_df'")
    print("="*80 + "\n")

    return df_final

# Execute the pipeline
bbqa_merged_df = load_and_merge_bbqa(config)

In [ ]:
# @title 7.5 MMLU Anchor Analytics: Knowledge Manifold Breadth - Configurable Anchor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

print("="*80)
print(" 🔬 ANALYZING MMLU KNOWLEDGE ANCHOR DISTRIBUTION")
print("="*80 + "\n")

# 1. Load MMLU Metadata
print(f"📥 Fetching MMLU subjects from {config.MMLU_DATASET}...")
mmlu_ds = load_dataset(config.MMLU_DATASET, split="train")

# ---------------------------------------------------------
# NEW FEATURE: CONFIGURABLE MMLU ANCHOR SIZE (mmlu_anchor_size)
# ---------------------------------------------------------
if hasattr(config, 'mmlu_data_size') and config.mmlu_data_size is not None:
    total_mmlu = len(mmlu_ds)
    if config.mmlu_data_size < total_mmlu:
        print(f"✂️ Applying MMLU Anchor Sampling: {config.mmlu_data_size} records...")
        # Deterministic sampling
        mmlu_ds = mmlu_ds.shuffle(seed=config.SEED).select(range(config.mmlu_data_size))
    else:
        print(f"ℹ️ Config 'mmlu_data_size' ({config.mmlu_data_size}) >= Available. Using full MMLU set.")

df_mmlu = pd.DataFrame(mmlu_ds)

# 2. Extract Top Subjects for Visualization
subject_counts = df_mmlu['subject'].value_counts().head(20)

# 3. Generating Visualization (Matching BBQ Style)
sns.set_theme(style="whitegrid", context="talk")
plt.figure(figsize=(14, 8))

ax = sns.barplot(
    x=subject_counts.values,
    y=subject_counts.index,
    palette="magma",
    edgecolor='black',
    linewidth=1.2
)

plt.title(f"MMLU Anchor Distribution: Top 20 Knowledge Domains\n(Total Active Anchor N = {len(df_mmlu):,})",
          fontsize=16, weight='bold', pad=20)
plt.xlabel("Count of Factual Samples (Labeled Strictly 1/Unbiased)", fontsize=13, weight='bold')
plt.ylabel("MMLU Subject Area", fontsize=13, weight='bold')

for p in ax.patches:
    width = p.get_width()
    ax.annotate(f'{int(width)}',
                (width, p.get_y() + p.get_height() / 2),
                xytext=(5, 0), textcoords='offset points',
                ha='left', va='center', fontsize=11, weight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ Knowledge Anchor Verified.")
print(f"   • Total MMLU Samples (Used): {len(df_mmlu):,}")
print(f"   • Unique Domains:            {df_mmlu['subject'].nunique()}")
print("   • Research Impact:           Ensures the BAD probe distinguishes 'Bias' from 'General Knowledge'.")
print("="*80 + "\n")

In [ ]:
# @title 7.6 Joint Manifold Balance: BBQ vs. MMLU Anchor
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# ═══════════════════════════════════════════════════════════════════════════
# 1. DYNAMIC DATA COUNTING (Aligning with config.num_bbq_samples)
# ═══════════════════════════════════════════════════════════════════════════
# We use the current state of bbq_merged_df which was sub-sampled in Cell 6
# to match your config.num_bbq_samples.
bbq_count = len(bbq_merged_df)
mmlu_count = len(df_mmlu)

# Validation check to ensure config compliance
if hasattr(config, 'num_bbq_samples') and config.num_bbq_samples is not None:
    # If the user requested fewer than available, the count should match config
    target_bbq = config.num_bbq_samples
    print(f"ℹ️  Configured BBQ Target: {target_bbq:,} samples.")
else:
    print(f"ℹ️  No sub-sampling requested. Using full BBQ manifold.")

labels = ['BBQ Bias Distribution', 'MMLU Knowledge Anchor']
sizes = [bbq_count, mmlu_count]
total_n = sum(sizes)
colors = ['#ff7675', '#0984e3'] # FairSteer Brand Colors: Red (Bias) vs Blue (Knowledge)
explode = (0, 0.1)  # Surgically separate the MMLU Anchor for visual emphasis

# ═══════════════════════════════════════════════════════════════════════════
# 2. GENERATE PROFESSIONAL MANIFOLD VISUALIZATION
# ═══════════════════════════════════════════════════════════════════════════
sns.set_theme(style="white")
fig, ax = plt.subplots(figsize=(10, 8), dpi=150)

patches, texts, autotexts = ax.pie(
    sizes,
    explode=explode,
    labels=labels,
    colors=colors,
    autopct=lambda p: f'{p:.1f}%\n({int(p * total_n / 100):,} samples)',
    shadow=True,
    startangle=140,
    textprops={'fontsize': 12, 'weight': 'bold'},
    pctdistance=0.75
)

# Maintain visual clarity for white/dark backgrounds
for autotext in autotexts:
    autotext.set_color('white')

# ═══════════════════════════════════════════════════════════════════════════
# 3. RESEARCH CONTEXT & SUMMARY
# ═══════════════════════════════════════════════════════════════════════════
plt.title(f"FairSteer Training Manifold: {config.base_model_name.split('/')[-1]}\n"
          f"(Unified Dataset N = {total_n:,})",
          fontsize=16, weight='bold', pad=20)

# Create a clean, scientific legend
plt.legend(
    patches,
    [f"{l}: {s:,}" for l, s in zip(labels, sizes)],
    title="Latent Sources",
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1),
    frameon=True,
    shadow=True
)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("📝 ARCHITECT'S MANIFOLD SUMMARY (CONFIG COMPLIANT)")
print("="*80)
print(f"• Config Load Limit:    {config.num_bbq_samples if config.num_bbq_samples else 'None':<10}")
print(f"• Active BBQ Samples:   {bbq_count:,}")
print(f"• Active MMLU Anchor:   {mmlu_count:,}")
print(f"• Combined Saturation:  {total_n:,} snapshots.")
print(f"• Anchor Strength:      {(mmlu_count/total_n):.1%} of the total training manifold.")
print("-" * 80)
print("• Causal Prediction:    The resulting BAD probe will distinguish between")
print("                         'Logical Certainty' (MMLU) and 'Social Bias' (BBQ).")
print("="*80 + "\n")

In [ ]:
# @title 8. Production-Grade Hook Manager (LLaDA Bidirectional Sniper Edition)
import torch
import numpy as np
from typing import List, Dict, Optional
import weakref

class MultiLayerHookManager:
    """
    LLaDA RESEARCH STANDARD: Masked-Mean Sniper.
    Surgically extracts activations ONLY from [MASK] token positions.
    Collapses bidirectional reasoning into a singular manifold via mean-pooling.
    """
    _active_managers = weakref.WeakSet()

    def __init__(self, model, layer_indices: List[int], mask_token_id: int):
        self.model = model
        self.layer_indices = sorted(layer_indices)
        self.mask_token_id = mask_token_id
        # Storage initialized on CPU for VRAM safety
        self.activations: Dict[int, torch.Tensor] = {l: None for l in layer_indices}
        self.hooks = []
        self._is_registered = False
        MultiLayerHookManager._active_managers.add(self)

    def _create_hook_fn(self, layer_idx: int):
        def hook_fn(module, input_args, output):
            """
            input_args[0] is typically the input_ids (batch, seq_len)
            output[0] is the residual stream (batch, seq_len, hidden_dim)
            """
            # 1. Identify Mask coordinates from the input
            # In LLaDA/Dream, input_args[0] contains the token IDs fed to the forward pass
            input_ids = input_args[0] 
            mask_binary = (input_ids == self.mask_token_id) # Shape: (batch, seq_len)

            # 2. Access residual stream
            h = output[0] if isinstance(output, tuple) else output # (batch, seq_len, hidden)

            # 3. LLaDA EXTRACTION LOGIC: Masked-Mean Pooling
            # We only care about the vectors where the model is predicting tokens.
            # We zero out non-masked positions and calculate the mean.
            masked_h = h * mask_binary.unsqueeze(-1).to(h.dtype)
            
            # Sum across sequence length and divide by number of masks per sequence
            sum_h = masked_h.sum(dim=1)
            count_masks = mask_binary.sum(dim=1, keepdim=True).clamp(min=1)
            mean_masked_h = sum_h / count_masks.to(h.dtype)

            # 4. SNIPER OFFLOAD: Move to CPU immediately to keep GPU 0.0% fragmented
            self.activations[layer_idx] = mean_masked_h.detach().clone().to('cpu', dtype=torch.float16)
            
        return hook_fn

    def get_batch_manifold(self) -> torch.Tensor:
        """Reconstructs the multi-layer manifold on the CPU."""
        return torch.stack([self.activations[l] for l in self.layer_indices], dim=1)

    def register(self):
        if self._is_registered: return
        for l in self.layer_indices:
            # Dream-7B / LLaDA architecture path
            target = self.model.model.layers[l]
            self.hooks.append(target.register_forward_hook(self._create_hook_fn(l)))
        self._is_registered = True

    def clear(self):
        """Wipe CPU references to prepare for next batch."""
        for l in self.layer_indices:
            self.activations[l] = None

    def remove(self):
        for h in self.hooks: h.remove()
        self.hooks = []; self._is_registered = False

    def __enter__(self): self.register(); return self
    def __exit__(self, *args): self.remove(); torch.cuda.empty_cache()

# MLP-Specific Sniper (Pointing to the FFN output instead of the total block output)
class MLPHookManager(MultiLayerHookManager):
    def register(self):
        if self._is_registered: return
        for l in self.layer_indices:
            # Dream-7B uses .mlp naming convention
            target = self.model.model.layers[l].mlp
            self.hooks.append(target.register_forward_hook(self._create_hook_fn(l)))
        self._is_registered = True

In [ ]:
# @title 9. Data Pipeline: Strict 1:1 Manifold Balancing - 1 Biased and 1 Unbiased mapping with GroupShuffleSplit

import numpy as np
from sklearn.model_selection import GroupShuffleSplit

def prepare_data_pipeline(labels, config):
    """
    Bytedance Standard: Strict 1 to 1 Undersampling.
    Ensures the probe learns the bias manifold, not class frequency.

    This revised version implements Group Based Stratification to prevent
    contextual identity leakage between training and validation sets.
    """

    # One. Identify indices for each pole
    idx_biased = np.where(labels == 0)[0]
    idx_neutral = np.where(labels == 1)[0]

    # Two. Find the minority count for the limiting factor
    n_samples_per_class = min(len(idx_biased), len(idx_neutral))

    # Sanity Gate: Ensure class presence
    if n_samples_per_class == 0:
        raise ValueError(f"Critical Error: One class has zero samples. "
                         f"Biased: {len(idx_biased)}, Neutral: {len(idx_neutral)}.")

    # Three. Deterministic Downsampling for parity
    # We use the seed for research reproducibility
    rng = np.random.default_rng(config.SEED)
    sampled_idx_biased = rng.choice(idx_biased, n_samples_per_class, replace=False)
    sampled_idx_neutral = rng.choice(idx_neutral, n_samples_per_class, replace=False)

    # Recombine balanced indices
    balanced_indices = np.concatenate([sampled_idx_biased, sampled_idx_neutral])

    # Four. Group Based Split Implementation
    # Requirement: group_ids must be accessible via config or global namespace
    # These represent the example_id or context identifiers from the BBQ dataset
    if not hasattr(config, "group_ids"):
        print("Warning: group_ids not found in config. Falling back to random split.")
        np.random.shuffle(balanced_indices)
        split_point = int(len(balanced_indices) * 0.8)
        train_set_idxs = balanced_indices[:split_point]
        val_set_idxs = balanced_indices[split_point:]
    else:
        # Extract groups for the balanced subset
        active_groups = config.group_ids[balanced_indices]
        active_labels = labels[balanced_indices]

        # Initialize the Group Based Stratification engine
        gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=config.SEED)

        # Perform the surgical split
        # This ensures context isolation between training and validation manifolds
        train_indices_rel, val_indices_rel = next(gss.split(balanced_indices, active_labels, active_groups))

        # Map back to original manifold coordinates
        train_set_idxs = balanced_indices[train_indices_rel]
        val_set_idxs = balanced_indices[val_indices_rel]

    # Five. Audit Logging
    print(f"FairSteer Manifold Audit (Grouped and Balanced):")
    print(f"   Biased Pole at zero: {n_samples_per_class} samples")
    print(f"   Neutral Pole at one: {n_samples_per_class} samples")
    print(f"   Ratio: 1 to 1")
    print(f"   Total Aligned Snapshots: {len(balanced_indices)}")
    print(f"   Split: {len(train_set_idxs)} Train | {len(val_set_idxs)} Val")
    print(f"   Status: Context leakage mitigated via Grouped Stratification")

    return train_set_idxs, val_set_idxs

In [ ]:
# @title 10. BAD Solver Library: Training Engine : sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import torch
import numpy as np
from tqdm.auto import tqdm

def train_probes(seed, train_set_idxs, val_set_idxs, activations, labels, num_layers):
    """
    Forensic Training Engine for Biased Activation Detection.
    Aligned with MIT standards for reproducibility and numerical stability.
    """
    all_accs = []
    all_bal_accs = []
    probes = []

    # Partitioning the manifold based on precalculated indices
    X_train_all = activations[train_set_idxs]
    X_val_all = activations[val_set_idxs]
    y_train = labels[train_set_idxs]
    y_val = labels[val_set_idxs]

    print("Commencing BAD Probe Training: Stability Mode Active")

    for layer in tqdm(range(num_layers), desc="Executing Layer Sweep"):
        # 1. Stability Upgrade: Cast to float32 for optimization stability
        X_train = X_train_all[:, layer, :].astype(np.float32)
        X_val = X_val_all[:, layer, :].astype(np.float32)

        # 2. Solver Configuration: Explicit L2 and Deterministic Seed
        # C is the inverse of regularization strength
        clf = LogisticRegression(
            C=1.0,
            solver="lbfgs",
            max_iter=10000,
            random_state=seed,
        )

        # 3. Model Fitting
        clf.fit(X_train, y_train)

        # 4. Forensic Evaluation
        y_val_pred = clf.predict(X_val)

        # Metric calculation
        acc = accuracy_score(y_val, y_val_pred)
        bal_acc = balanced_accuracy_score(y_val, y_val_pred)

        # Storage
        all_accs.append(acc)
        all_bal_accs.append(bal_acc)
        probes.append(clf)

        # Observability
        print(f"   Success L{layer:02d} | Balanced Acc: {bal_acc:.2%}")

    # Return lists converted to numpy arrays for downstream processing
    return probes, np.array(all_accs), np.array(all_bal_accs)

In [ ]:
# @title 10.5: FairSteer Tokenization & Prompt Construction (LLaDA Diffusion Engine)
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch.nn.functional as F

def apply_llada_masking(token_ids: torch.Tensor, prompt_len: int, mask_token_id: int, t: float = 0.5):
    """
    LLaDA Masking Engine: Implements Algorithm 2 (SFT) masking protocol.
    Leaves the prompt unmasked, and masks the response with probability t.
    """
    # Clone to avoid modifying the original tensor
    masked_ids = token_ids.clone()
    seq_len = token_ids.shape[-1]
    
    # Generate random mask for the response portion only
    # indices [prompt_len : seq_len]
    response_len = seq_len - prompt_len
    if response_len <= 0:
        return masked_ids # No response to mask
        
    mask_dist = torch.full((1, response_len), t)
    mask_binary = torch.bernoulli(mask_dist).to(torch.bool)
    
    # Apply [MASK] token
    masked_ids[0, prompt_len:][mask_binary[0]] = mask_token_id
    
    return masked_ids

def tokenized_bbqa(dataset, tokenizer, config):
    """
    TEACHER-FORCING ENGINE (Diffusion): 
    Generates [MASK]ed scenarios for high-precision DSV computation.
    """
    all_prompts, all_labels = [], []
    choice_map = ['A', 'B', 'C']

    print(f"🚀 Distilling LLaDA BBQA Manifold (t={config.extraction_t})")

    for _, row in tqdm(dataset.iterrows(), total=len(dataset), desc="Processing BBQA"):
        label = int(row['label'])
        target_loc = int(row['target_loc'])

        if pd.isna(target_loc) or label == target_loc:
            continue

        ctx = row['context']
        q = row['question']
        options = f"\nA. {row['ans0']}\nB. {row['ans1']}\nC. {row['ans2']}\nAnswer:"
        
        prompt_text = f"{ctx} {q}"
        prompt_ids = tokenizer(prompt_text, return_tensors='pt').input_ids
        p_len = prompt_ids.shape[-1]

        # Scenario Logic: (Neutral Pole 1, Biased Pole 0)
        scenarios = [(label, 1), (target_loc, 0)]

        for choice_idx, lab in scenarios:
            full_text = f"{prompt_text}{options} {choice_map[choice_idx]}"
            full_ids = tokenizer(full_text, return_tensors='pt').input_ids
            
            # Apply LLaDA Stochastic Masking to the Response portion
            masked_input = apply_llada_masking(
                full_ids, p_len, config.mask_token_id, t=config.extraction_t
            )

            all_prompts.append(masked_input)
            all_labels.append(lab)

    return all_prompts, all_labels

def tokenized_bbq_behavioral(dataset, tokenizer, model, config, few_shot=False):
    """
    BEHAVIORAL ENGINE (Diffusion): 
    Uses Conditional Likelihood (Eq 6) to determine if Dream-7B is biased.
    """
    all_prompts, all_labels = [], []
    choice_map = ["A", "B", "C"]
    
    # Pre-resolve token IDs for A, B, C
    choice_token_ids = [tokenizer.convert_tokens_to_ids(c) for c in choice_map]

    print(f"🔬 LLaDA Behavioral Audit: {'Few-Shot' if few_shot else 'Zero-Shot'}")

    for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
        prompt_text = f"{row['context']} {row['question']}\nA. {row['ans0']}\nB. {row['ans1']}\nC. {row['ans2']}\nAnswer:"
        if few_shot: prompt_text = config.BBQ_FS_DATA + prompt_text
        
        prompt_ids = tokenizer(prompt_text, return_tensors='pt').input_ids.to(model.device)
        
        # In Diffusion, we evaluate bias by checking which choice is more likely
        # We append a [MASK] at the end and check model prediction
        mask_input = torch.cat([prompt_ids, torch.tensor([[config.mask_token_id]]).to(model.device)], dim=1)
        
        with torch.no_grad():
            outputs = model(input_ids=mask_input)
            # Logits for the MASK position (last index)
            mask_logits = outputs.logits[0, -1, :]
            
            # Extract probabilities for A, B, C
            probs = F.softmax(mask_logits[choice_token_ids], dim=0)
            pred = torch.argmax(probs).item()

        label = None
        if pred == int(row['label']):
            label = 1 # Neutral Behavior
        elif pred == int(row['target_loc']):
            label = 0 # Biased Behavior

        if label is not None:
            # For extraction, we provide the model with a mid-step diffusion state (t=0.5)
            extract_input = apply_llada_masking(
                mask_input.cpu(), prompt_ids.shape[-1], config.mask_token_id, t=config.extraction_t
            )
            all_prompts.append(extract_input)
            all_labels.append(label)

    return all_prompts, all_labels

def tokenized_mmlu(dataset, tokenizer, config):
    """KNOWLEDGE ANCHOR ENGINE: LLaDA Standard."""
    all_prompts, all_labels = [], []

    for _, row in tqdm(dataset.iterrows(), total=len(dataset), desc="Processing MMLU"):
        if int(row['answer']) >= 3: continue # Parity with BBQ A/B/C

        prompt_text = f"{row['question']}"
        prompt_ids = tokenizer(prompt_text, return_tensors='pt').input_ids
        
        response_text = f"\nA. {row['choices'][0]}\nB. {row['choices'][1]}\nC. {row['choices'][2]}\nAnswer:"
        full_text = prompt_text + response_text
        full_ids = tokenizer(full_text, return_tensors='pt').input_ids

        # Masking the Response portion
        masked_input = apply_llada_masking(
            full_ids, prompt_ids.shape[-1], config.mask_token_id, t=config.extraction_t
        )

        all_prompts.append(masked_input)
        all_labels.append(1) # Knowledge is always Neutral Anchor

    return all_prompts, all_labels

In [ ]:
# @title 11. Master Manifold Distiller: Part 1 - LLaDA Causal Extraction (A100/MPS Optimized)
import os, gc, torch, numpy as np, pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm.auto import tqdm
from numpy.lib.format import open_memmap

# ═══════════════════════════════════════════════════════════════
# 1. HARDWARE & ARCHITECTURAL INITIALIZATION
# ═══════════════════════════════════════════════════════════════
model_id = config.model_id_short
layers = sorted(config.candidate_layers_range)

# OpenAI Standard: Resource partitioning for LLaDA Manifolds
DSV_LIMIT = config.DSV_TARGET
BEHAVIORAL_LIMIT = config.num_bbq_samples 

# 1. Initialize the Dream-7B Manifold
# LLaDA uses SDPA to handle dense bidirectional attention efficiently
base_model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name, 
        torch_dtype=torch.float16,
        device_map="auto", 
        attn_implementation="sdpa" 
    ).eval()

# 2. Dynamic Mask ID Resolution (Standard Guardrail)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name, use_fast=False)
if tokenizer.mask_token_id is not None:
    config.mask_token_id = tokenizer.mask_token_id
else:
    config.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

print(f"✅ LLaDA Architecture Active: [MASK] ID = {config.mask_token_id}")

# Google Standard: LLaDA works with simple padding as it is bidirectional
tokenizer.padding_side = "right" 
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token

# 3. Initialize Hook Managers with Masked-Mean Pooling
res_manager = MultiLayerHookManager(base_model, layers, config.mask_token_id)

# MLP-Specific addressing for Dream-7B
class DreamMLPHookManager(MultiLayerHookManager):
    def register(self):
        if self._is_registered: return
        for l in self.layer_indices:
            target = self.model.model.layers[l].mlp
            self.hooks.append(target.register_forward_hook(self._create_hook_fn(l)))
        self._is_registered = True

mlp_manager = DreamMLPHookManager(base_model, layers, config.mask_token_id)

# ═══════════════════════════════════════════════════════════════
# 2. PHASE-AWARE EXTRACTION ENGINE
# ═══════════════════════════════════════════════════════════════
modes = [
    {'name': 'bbqa',   'type': 'forced',     'limit': DSV_LIMIT},
    {'name': 'bbq_zs', 'type': 'behavioral', 'limit': BEHAVIORAL_LIMIT, 'fs': False},
    {'name': 'bbq_fs', 'type': 'behavioral', 'limit': BEHAVIORAL_LIMIT, 'fs': True},
    {'name': 'mmlu',   'type': 'anchor',     'limit': config.mmlu_data_size}
]

for m in modes:
    print(f"\n▶️ LLaDA DISTILLATION START: {m['name'].upper()}")

    # [A] Load Data via LLaDA stochastic masking functions (from Cell 10.5)
    if m['name'] == 'bbqa':
        prompts, labels = tokenized_bbqa(bbqa_merged_df, tokenizer, config)
    elif m['name'] == 'mmlu':
        mmlu_local = pd.DataFrame(load_dataset(config.MMLU_DATASET, split="train").shuffle(seed=42).select(range(m['limit'])))
        prompts, labels = tokenized_mmlu(mmlu_local, tokenizer, config)
    else:
        # Slicing the merged BBQ set to the Behavioral robustness target
        prompts, labels = tokenized_bbq_behavioral(bbq_merged_df.iloc[:m['limit']], tokenizer, base_model, config, few_shot=m['fs'])

    # [B] Prepare SSD-Direct Allocation (Memmap)
    base_out = f'activations/{m["name"]}/{model_id}'
    os.makedirs(base_out, exist_ok=True)
    path_layer = f'{base_out}/layer_wise.npy.tmp'
    path_mlp = f'{base_out}/mlp_wise.npy.tmp'

    # Shape: (Num_Samples, Num_Layers, Hidden_Dim)
    fp_layer = open_memmap(path_layer, mode='w+', dtype='float16', shape=(len(prompts), len(layers), 4096))
    fp_mlp = open_memmap(path_mlp, mode='w+', dtype='float16', shape=(len(prompts), len(layers), 4096))

    valid_count = 0
    
    try:
        with res_manager, mlp_manager:
            for i in tqdm(range(0, len(prompts), config.extraction_batch_size), desc="Denoising Forward Pass"):
                chunk = prompts[i : i + config.extraction_batch_size]
                # Bidirectional models require correct padding for batch alignment
                batch = tokenizer.pad({"input_ids": [p.squeeze(0) for p in chunk]}, return_tensors="pt").to(device)

                with torch.inference_mode():
                    # Full sequence forward pass (No KV cache optimization possible)
                    base_model(**batch)

                # 🚀 COLLECT MASKED-MEAN SNAPSHOTS (Direct from CPU Sniper Hooks)
                res_cpu_batch = res_manager.get_batch_manifold() # Shape: [Batch, Layers, 4096]
                mlp_cpu_batch = mlp_manager.get_batch_manifold()

                n_actual = len(chunk)
                fp_layer[valid_count : valid_count + n_actual] = res_cpu_batch.numpy()
                fp_mlp[valid_count : valid_count + n_actual] = mlp_cpu_batch.numpy()

                valid_count += n_actual

                # RECLAMATION (OpenAI Standard)
                res_manager.clear(); mlp_manager.clear()
                del batch, res_cpu_batch, mlp_cpu_batch
                if i % 10 == 0: gc.collect(); torch.cuda.empty_cache()

    except Exception as e:
        print(f"❌ Critical Failure during {m['name']}: {e}"); raise

    # [C] SOLIDIFICATION
    print(f"🔄 Finalizing Manifold: {m['name']} ({valid_count} samples)")
    np.save(f'{base_out}/labels.npy', np.array(labels[:valid_count]))
    np.save(f'{base_out}/layer_wise.npy', np.array(fp_layer[:valid_count]))
    np.save(f'{base_out}/mlp_wise.npy', np.array(fp_mlp[:valid_count]))

    fp_layer._mmap.close(); fp_mlp._mmap.close()
    if os.path.exists(path_layer): os.remove(path_layer)
    if os.path.exists(path_mlp): os.remove(path_mlp)

print("\n" + "="*80 + "\n🏁 LLaDA MANIFOLD DISTILLATION COMPLETE\n" + "="*80)

In [ ]:
# @title 11.3: Manifold Merger - Consolidating the BAD Training Set
import numpy as np
import os, gc

model_id = config.base_model_name.split("/")[-1]
# BBQA is excluded (it is for DSV calculation, not probe training)
source_datasets = ['bbq_zs', 'bbq_fs', 'mmlu']
target_dir = f'activations/probes/{model_id}'
os.makedirs(target_dir, exist_ok=True)

print("="*80)
print(f" 🧩 CONSOLIDATING UNIFIED MANIFOLD: {model_id}")
print("   Mechanism: list.extend (FairSteer GitHub Standard)")
print("="*80 + "\n")

# Initialize empty lists exactly as FairSteer does
all_layer_wise_activations = []
all_mlp_wise_activations = []
all_labels = []

for ds in source_datasets:
    path = f"activations/{ds}/{model_id}"

    # Defensive check for directory existence
    if os.path.exists(f"{path}/layer_wise.npy"):
        # Load the sub-manifold
        l_act = np.load(f"{path}/layer_wise.npy")
        m_act = np.load(f"{path}/mlp_wise.npy")
        lbls = np.load(f"{path}/labels.npy")

        # PROOF: Using list.extend(ndarray)
        # This iterates over the sample dimension (axis 0) and adds each
        # [Layers, Dim] snapshot as an element in the list.
        # We cast to float16/int16 during the extend to maintain SSD safety.
        # all_layer_wise_activations.extend(l_act.astype(np.float16))
        # all_mlp_wise_activations.extend(m_act.astype(np.float16))
        # all_labels.extend(lbls.astype(np.int16))
        # Extending lists using native precision and types to match the reference code
        all_layer_wise_activations.extend(l_act)
        all_mlp_wise_activations.extend(m_act)
        all_labels.extend(lbls)

        # PROOF: Observability prints matching original FS logic
        print(f"   ✓ Extended with {ds:<8}: all_layer_wise_activations = {len(all_layer_wise_activations):>6}")
        print(f"   ✓ Extended with {ds:<8}: all_labels Length = {len(all_labels):>6}")
        print(f"   ✓ Extended with {ds:<8}: all_mlp_wise_activations Length = {len(all_mlp_wise_activations):>6}")
        # Cleanup temporary local pointers to free RAM for the next dataset
        del l_act, m_act, lbls
        gc.collect()
    else:
        print(f"   ⚠️  Skipping {ds}: Data not found.")

# ═══════════════════════════════════════════════════════════════════════════
# FINAL SOLIDIFICATION (Saving the aggregated lists)
# ═══════════════════════════════════════════════════════════════════════════
if not all_labels:
    print("❌ CRITICAL FAILURE: No data found to merge.")
else:
    print(f"\n💾 Saving Consolidated Manifold to {target_dir}...")

    # np.save handles the list-of-arrays to ndarray conversion internally
    np.save(f'{target_dir}/layer_wise.npy', all_layer_wise_activations)
    np.save(f'{target_dir}/mlp_wise.npy', all_mlp_wise_activations)
    np.save(f'{target_dir}/labels.npy', all_labels)

    print(f"\n✅ SUCCESS: Manifold Recombined.")
    print(f"   • Final Count: {len(all_labels):,} samples.")

    # Total cleanup
    del all_layer_wise_activations, all_mlp_wise_activations, all_labels
    gc.collect()

print("="*80)

In [ ]:
# @title 11.4: Steering Vector (DSV) Calculation - LLaDA Normalized Centroid Mapping
import numpy as np
import os, gc

def synthesize_llada_dsv_manifolds():
    """
    LLaDA CORE LOGIC: Equation 3 (Refined for Diffusion)
    Surgically calculates unit-normalized steering directions from the 
    Masked-Mean pooled BBQA manifold.
    """
    print("="*80)
    print(f" 🧪 LLaDA DSV CALCULATOR: Unit-Normalized Centroid Mapping")
    print(f"   Architecture: Dream-7B Bidirectional Diffusion")
    print("="*80 + "\n")
    
    model_id = config.model_id_short
    input_path = f"activations/bbqa/{model_id}"

    if not os.path.exists(f"{input_path}/labels.npy"):
        raise FileNotFoundError(f"❌ Error: BBQa data not found. Run Distiller Pass 1 first.")

    # 1. LOAD MASKED-MEAN SNAPSHOTS
    # These are already pooled averages of the [MASK] positions
    labels = np.load(f"{input_path}/labels.npy")
    layer_acts = np.load(f"{input_path}/layer_wise.npy", mmap_mode='r')
    mlp_acts = np.load(f"{input_path}/mlp_wise.npy", mmap_mode='r')

    num_samples, num_layers, hidden_dim = layer_acts.shape
    print(f"   ✓ Input Manifold: {num_samples} pooled snapshots | {num_layers} Layers")

    def compute_normalized_centroid_delta(wise_activations, labels):
        steering_vectors = []
        
        for l in range(num_layers):
            # PRECISION GUARD: We use float64 for centroid math to prevent 
            # floating point drift in the 4096-dim space.
            layer_data = wise_activations[:, l, :].astype(np.float64)

            # POLE 1: Neutral Centroid (Label 1)
            mu_neutral = np.mean(layer_data[labels == 1], axis=0)
            
            # POLE 0: Biased Centroid (Label 0)
            mu_biased = np.mean(layer_data[labels == 0], axis=0)

            # DIRECTION: Biased -> Neutral shift
            direction = mu_neutral - mu_biased
            
            # 🚨 UNIT-NORMALIZATION GUARD (Standard for Diffusion Steering)
            # This ensures that alpha control is mathematically linear.
            norm = np.linalg.norm(direction)
            if norm > 1e-6:
                unit_direction = direction / norm
            else:
                unit_direction = direction # Avoid division by zero
                
            steering_vectors.append(unit_direction)

            if l % 10 == 0: gc.collect()

        return np.array(steering_vectors)

    # 2. CALCULATE STEERING MAPS
    print("🔄 Mapping Residual Stream Steering Hyperplane...")
    layer_dsv = compute_normalized_centroid_delta(layer_acts, labels)

    print("🔄 Mapping MLP Manifold Steering Hyperplane...")
    mlp_dsv = compute_normalized_centroid_delta(mlp_acts, labels)

    # 3. ARTIFACT ARCHIVING
    save_dir = 'vectors'
    os.makedirs(save_dir, exist_ok=True)
    np.save(f'{save_dir}/{model_id}_layer_wise.npy', layer_dsv.astype(np.float32))
    np.save(f'{save_dir}/{model_id}_mlp_wise.npy', mlp_dsv.astype(np.float32))

    # 4. SIGNAL AUDIT
    # We audit Layer 15 (typical semantic bottleneck for 7B models)
    audit_layer = 15
    audit_norm = np.linalg.norm(layer_dsv[audit_layer])
    print(f"\n💾 STEERING VECTORS SECURED to /{save_dir}/")
    print(f"🔍 [SIGNAL AUDIT] Layer {audit_layer} Unit-Norm: {audit_norm:.4f}")
    print("✅ Normalization Guard: ACTIVE.")
    print("="*80 + "\n")

    return {"layer": layer_dsv, "mlp": mlp_dsv}

# Execute calculation
dsv_manifold = synthesize_llada_dsv_manifolds()
gc.collect()

In [ ]:
# @title 11.5 Master Manifold Distiller: Part 2 - LLaDA BAD Training & Causal Packaging
import os, torch, joblib, gc
from datetime import datetime
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# ═══════════════════════════════════════════════════════════════
# 1. MANIFOLD PREPARATION (Diffusion-Aware)
# ═══════════════════════════════════════════════════════════════
model_id = config.base_model_name.split("/")[-1]
probe_data_path = f"activations/probes/{model_id}"

print(f"📥 Loading Masked-Mean Manifolds: {probe_data_path}")

# Load the 3D manifold [Samples, Layers, Hidden_Dim]
all_X = np.load(f"{probe_data_path}/layer_wise.npy")
all_y = np.load(f"{probe_data_path}/labels.npy")

num_samples, num_layers, hidden_dim = all_X.shape
print(f"   ✓ Manifold Context: {num_samples} pooled snapshots | {num_layers} Layers | t={config.extraction_t}")

# Generate balanced indices (Ensures probe doesn't overfit to the Neutral anchor)
train_set_idxs, val_set_idxs = prepare_data_pipeline(all_y, config)

# ═══════════════════════════════════════════════════════════════
# 2. DIFFUSION PROBE SOLVER (L-BFGS High-Precision)
# ═══════════════════════════════════════════════════════════════
layer_probes = []
val_metrics = []

print(f"\n🚀 Training Diffusion Watchmen (L0 - L{num_layers-1})")

for l_idx in tqdm(range(num_layers), desc="Layer-wise L-BFGS Sweep"):
    # 1. Extraction: Move specific layer to float32 for optimization stability
    X_train = all_X[train_set_idxs, l_idx, :].astype(np.float32)
    y_train = all_y[train_set_idxs]
    
    X_val = all_X[val_set_idxs, l_idx, :].astype(np.float32)
    y_val = all_y[val_set_idxs]

    # 2. Solver Initialization: 
    # C=1.0 is standard; high max_iter to handle the dense bidirectional space
    clf = LogisticRegression(
        C=1.0,
        solver="lbfgs",
        max_iter=10000, 
        random_state=config.SEED,
        n_jobs=-1 # Utilize all CPU cores for the linear solve
    )

    # 3. Fit & Evaluate
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    
    acc = accuracy_score(y_val, y_pred)
    bal_acc = balanced_accuracy_score(y_val, y_pred)
    
    layer_probes.append(clf)
    val_metrics.append({'acc': acc, 'bal_acc': bal_acc})

# ═══════════════════════════════════════════════════════════════
# 3. CAUSAL KIT PACKAGING (The 'Watchman' Payload)
# ═══════════════════════════════════════════════════════════════
checkpoints_dir = os.path.join(config.local_save_dir, "checkpoints")
os.makedirs(checkpoints_dir, exist_ok=True)

# Load the Steering Vectors (computed in Cell 11.4)
dsv_path = f'vectors/{model_id}_layer_wise.npy'
dsv_manifold = np.load(dsv_path)

print(f"\n📦 Securing Causal Kits for Iterative Steering...")

for l_idx in range(num_layers):
    clf = layer_probes[l_idx]
    
    # Surgical Payload for the DAS Engine
    payload = {
        'model_state_dict': {
            'linear.weight': torch.from_numpy(clf.coef_).float(), # [1, 4096]
            'linear.bias': torch.from_numpy(clf.intercept_).float() # [1]
        },
        'layer_idx': l_idx,
        'extraction_t': config.extraction_t, # 🚨 CRITICAL: Link to diffusion timestep
        'mean_diff_vector': torch.from_numpy(dsv_manifold[l_idx]).float(), # Unit-Normalized DSV
        'metrics': val_metrics[l_idx],
        'timestamp': datetime.now().isoformat(),
        'model_id': model_id
    }

    # Save as specific layer artifact
    save_path = os.path.join(checkpoints_dir, f"{model_id}_BAD_L{l_idx}_t{config.extraction_t}.pt")
    torch.save(payload, save_path)

# 4. FINAL EXPORT & AUDIT
joblib.dump(layer_probes, f'probes/{model_id}_layer_wise.pkl')
np.save(f'probes/{model_id}_layer_wise_accuracy.npy', [m['acc'] for m in val_metrics])

peak_layer = np.argmax([m['bal_acc'] for m in val_metrics])
print(f"\n🏁 DIFFUSION TRAINING COMPLETE.")
print(f"   • Peak Balanced Acc: {val_metrics[peak_layer]['bal_acc']:.2%} at Layer {peak_layer}")
print(f"   • Checkpoint Path:   {checkpoints_dir}")
print(f"   • Status:            Causal Kits Ready for DAS Inference.")

# Memory Reclamation
del all_X, all_y; gc.collect(); torch.cuda.empty_cache()

In [ ]:
# @title 12. Phase 2: LLaDA Performance Visualization & Causal Audit (A100/MPS)
import gc, torch, os, json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# ═══════════════════════════════════════════════════════════════
# 1. ARTIFACT AUDIT (Loading the Diffusion Payloads)
# ═══════════════════════════════════════════════════════════════
checkpoints_dir = os.path.join(config.local_save_dir, "checkpoints")
model_id_short = config.model_id_short
layer_results = []

print(f"🔬 Auditing Diffusion Manifold for {model_id_short} at t={config.extraction_t}...")

# LLaDA/Dream-7B typically scans layers 0-31
for l in config.candidate_layers_range:
    # Using the LLaDA-specific naming convention from Cell 11.5
    filename = f"{model_id_short}_BAD_L{l}_t{config.extraction_t}.pt"
    path = os.path.join(checkpoints_dir, filename)
    
    if os.path.exists(path):
        # weights_only=False because we are loading dicts with tensors and floats
        ckpt = torch.load(path, map_location='cpu', weights_only=False)

        # Extract metrics and signal metadata
        metrics = ckpt.get('metrics', {})
        dsv = ckpt.get('mean_diff_vector')
        
        # Calculate L2 Norm of the unit-normalized vector (should be ~1.0)
        # but we track the raw separation distance if available
        norm = torch.norm(dsv).item() if dsv is not None else 0

        layer_results.append({
            'layer': l,
            'acc': metrics.get('acc', 0),
            'bal_acc': metrics.get('bal_acc', 0),
            'signal_strength': norm,
            't': ckpt.get('extraction_t', 0.5)
        })

if not layer_results:
    raise FileNotFoundError("❌ No LLaDA checkpoints found. Ensure Cell 11.5 completed successfully.")

df_plot = pd.DataFrame(layer_results).sort_values('layer')
best_l_idx = df_plot['bal_acc'].idxmax()
best_layer = int(df_plot.loc[best_l_idx]['layer'])
best_bal_acc = df_plot.loc[best_l_idx]['bal_acc']

# ═══════════════════════════════════════════════════════════════
# 2. DUAL-AXIS PUBLICATION PLOT (Standard Research Style)
# ═══════════════════════════════════════════════════════════════
sns.set_theme(style="white", context="paper")
fig, ax1 = plt.subplots(figsize=(14, 7), dpi=200)

# AXIS 1: Detection Selectivity (Primary Metric)
color_acc = '#0984e3' # OpenAI Blue
ax1.set_xlabel('Transformer Layer Index (LLaDA Depth)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Probe Balanced Accuracy', color=color_acc, fontsize=14, fontweight='bold')
lns1 = ax1.plot(df_plot['layer'], df_plot['bal_acc'], marker='o', markersize=8,
                linewidth=4, color=color_acc, label='Balanced Acc (Watchman)', zorder=4)
ax1.tick_params(axis='y', labelcolor=color_acc)
ax1.set_ylim(0.45, 1.05)
ax1.axhline(y=0.5, color='black', linestyle=':', alpha=0.3, label='Chance (0.5)')

# AXIS 2: Diffusion Signal Power
ax2 = ax1.twinx()
color_norm = '#2d3436' # Charcoal
ax2.set_ylabel('Steering Power (L2 Norm)', color=color_norm, fontsize=14, fontweight='bold')
lns2 = ax2.plot(df_plot['layer'], df_plot['signal_strength'], marker='s', markersize=6,
                linewidth=2, linestyle='--', color=color_norm, label='DSV Magnitude', alpha=0.6)
ax2.tick_params(axis='y', labelcolor=color_norm)

# ═══════════════════════════════════════════════════════════════
# 3. ANNOTATIONS & MECHANISTIC REGIONS (Diffusion Context)
# ═══════════════════════════════════════════════════════════════
# Highlight the Optimal Steering Point
ax1.scatter([best_layer], [best_bal_acc], color='gold', s=450, marker='*',
            edgecolor='black', zorder=5, label=f'Optimal Bottleneck L{best_layer}')

# Annotate the Diffusion Reasoning Stages
ax1.fill_between([0, 6], 0, 1.1, color='gray', alpha=0.05, label='Context Encoding')
ax1.fill_between([6, 20], 0, 1.1, color='green', alpha=0.05, label='Causal Bottleneck (Denoising)')
ax1.fill_between([20, 31], 0, 1.1, color='red', alpha=0.05, label='Token Crystallization')

plt.title(f"Dream-7B Diffusion Causal Bottleneck Profile (t={config.extraction_t})", 
          fontsize=18, fontweight='bold', pad=20)
ax1.grid(axis='y', linestyle='-', alpha=0.2)

# Unified Legend
lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='lower right', frameon=True, shadow=True)

plt.tight_layout()
plt.show()

# ═══════════════════════════════════════════════════════════════
# 4. ARCHITECT'S VERDICT
# ═══════════════════════════════════════════════════════════════
print(f"\n{'='*60}")
print(f"🏆 LLaDA CAUSAL WINNER: LAYER {best_layer}")
print(f"{'='*60}")
print(f"📊 Forensic Report for Dream-7B:")
print(f"   • Peak Balanced Accuracy: {best_bal_acc:.2%}")
print(f"   • Diffusion Step Audited: t={config.extraction_t}")
print(f"   • Signal Integrity:       {'OPTIMAL' if best_bal_acc > 0.85 else 'SUBOPTIMAL'}")
print(f"\n🧠 Insight:")
print(f"   In bidirectional diffusion, bias detection peaks at Layer {best_layer}.")
print(f"   Applying DAS at this layer will yield the highest 'debiasing per unit of alpha'.")
print(f"{'='*60}\n")

# Assign global l_star for the next notebook (Inference/Eval)
globals()['l_star'] = best_layer
globals()['best_score'] = best_bal_acc

In [ ]:
# @title 14. Publication Figure: Layer-wise Causal Distillation Profile
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd

print("\n" + "="*80)
print(" 📈 GENERATING NEURIPS-STANDARD LAYER SENSITIVITY PLOT")
print("   Metric: Linear Separability of Bias Manifold")
print("="*80 + "\n")

# 1. DATA EXTRACTION
target_data = globals().get('layer_results') or globals().get('layer_summary')
if target_data is None:
    raise ValueError("❌ 'layer_results' not found. Ensure Cell 12 executed successfully.")

df_plot = pd.DataFrame(target_data).sort_values('layer')

# 2. CREATE PUBLICATION FIGURE
sns.set_theme(style="white", context="paper", font_scale=1.4)
fig, ax = plt.subplots(figsize=(12, 7), dpi=300)

# 🚨 IMPROVEMENT: Mechanistic Phase Shading
# Highlights the window where the model is most 'malleable'
ax.axvspan(0, 8, color='gray', alpha=0.05, label='Syntactic Stage')
ax.axvspan(8, 22, color='#2ecc71', alpha=0.05, label='Causal Bottleneck (Semantic)')
ax.axvspan(22, 31, color='#e74c3c', alpha=0.05, label='Decision Commitment')

# A. Plot Metrics
# Primary Metric: Balanced Accuracy (The true measure of bias detection)
ax.plot(df_plot['layer'], df_plot['bal_acc'],
        color='#4285F4', label='Detection Selectivity (M2)',
        linewidth=4, marker='o', markersize=10, zorder=5)

# Secondary Metric: Standard Accuracy (Model confidence)
if 'std_acc' in df_plot.columns:
    ax.plot(df_plot['layer'], df_plot['std_acc'],
            color='#34A853', label='Standard Logit Acc (M1)',
            linewidth=2, linestyle='--', alpha=0.6, zorder=4)

# B. Shaded 'Chance' Region (50% is random guess)
ax.axhline(y=0.5, color='#d63031', linestyle=':', linewidth=2.5, label='Chance Baseline', zorder=1)
ax.fill_between(df_plot['layer'], 0, 0.5, color='#dfe6e9', alpha=0.3)

# C. Highlight Peak (Optimal Layer Selection)
best_acc = df_plot['bal_acc'].max()
best_l = df_plot.loc[df_plot['bal_acc'].idxmax()]['layer']
ax.scatter(best_l, best_acc, color='#D63031', marker='*', s=600, zorder=6,
           edgecolor='black', label=f'Optimal Bottleneck L{int(best_l)}')

# D. Refined Annotation
ax.annotate(
    f"Peak Selectivity: {best_acc:.1%}",
    xy=(best_l, best_acc),
    xytext=(best_l - 6, best_acc + 0.05),
    ha='center', fontsize=13, fontweight='bold',
    bbox=dict(boxstyle='round,pad=0.5', fc='white', ec='#4285F4', lw=2, alpha=0.9),
    arrowprops=dict(arrowstyle='-|>', connectionstyle="arc3,rad=-0.2", color='#4285F4', lw=2)
)

# 3. STYLING & EXPORT
model_short = config.base_model_name.split("/")[-1]
ax.set_title(f"Bias Manifold Crystallization Profile: {model_short}", fontsize=20, fontweight='bold', pad=30)
ax.set_xlabel("Transformer Layer Index (Depth)", fontsize=16, fontweight='bold')
ax.set_ylabel("Probe Performance (Balanced Accuracy)", fontsize=16, fontweight='bold')

# Limits and Ticks
ax.set_ylim([0.45, 1.05])
ax.set_xticks(np.arange(0, 32, 2)) # Cleaner X-axis
ax.grid(axis='y', linestyle='--', alpha=0.3)

# Legend refinement
ax.legend(loc='lower right', frameon=True, shadow=True, facecolor='white', edgecolor='black', fontsize=11)

sns.despine(trim=True)
plt.tight_layout()

# 🚨 IMPROVEMENT: Save as PNG and PDF (Vector Format)
save_path_base = os.path.join(config.local_save_dir, f"layer_sensitivity_{model_short}")
plt.savefig(f"{save_path_base}.png", dpi=300, bbox_inches='tight')
plt.savefig(f"{save_path_base}.pdf", bbox_inches='tight')

plt.show()

print(f"✅ Publication-quality figures (PNG/PDF) secured in {config.local_save_dir}")

In [ ]:
#@title Clear Memory
import torch
import gc


# Clear Python garbage collection
gc.collect()

# Clear MPS cache (Apple GPU memory)
torch.mps.empty_cache()
